# Task 2 - Data Exploration, Analysis, and Preprocessing 

This notebook covers data quality, integration, comprehensive exploration, and preparing the data for the modeling tasks.

## 2.1 Setup and Data Loading

In [1]:
# import necessary libraries
import pandas as pd
import numpy as np

# Load the two primary datasets (Power Generation and Sensor Data for each plant)
plant1_gen = pd.read_csv('../data/raw/Plant_1_Generation_Data.csv')
plant1_weather = pd.read_csv('../data/raw/Plant_1_Weather_Sensor_Data.csv')
plant2_gen = pd.read_csv('../data/raw/Plant_2_Generation_Data.csv')
plant2_weather = pd.read_csv('../data/raw/Plant_2_Weather_Sensor_Data.csv')

## 2.2 Data Quality and Integration

### 2.2.1 Data Quality Assessment

#### A. General checks

##### Data Duplicates

In [2]:
# Check for row duplicates in each dataset
print("Plant 1 Generation duplicates:", plant1_gen.duplicated().sum(), f"({round(plant1_gen.duplicated().sum()/len(plant1_gen)*100, 2)}%)")
print("Plant 2 Generation duplicates:", plant2_gen.duplicated().sum(), f"({round(plant2_gen.duplicated().sum()/len(plant2_gen)*100, 2)}%)")
print("Plant 1 Weather duplicates:", plant1_weather.duplicated().sum(), f"({round(plant1_weather.duplicated().sum()/len(plant1_weather)*100, 2)}%)")
print("Plant 2 Weather duplicates:", plant2_weather.duplicated().sum(), f"({round(plant2_weather.duplicated().sum()/len(plant2_weather)*100, 2)}%)")

Plant 1 Generation duplicates: 911114 (89.22%)
Plant 2 Generation duplicates: 1291888 (90.9%)
Plant 1 Weather duplicates: 0 (0.0%)
Plant 2 Weather duplicates: 0 (0.0%)


In [3]:
# Constants for 34 Days @ 15-min intervals
DAYS = 34
READINGS_PER_DAY = 24 * 4 # 96
EXPECTED_PER_SENSOR = DAYS * READINGS_PER_DAY

def analyse_dataset(df, plant_label="Plant 1", days=DAYS, readings_per_day=READINGS_PER_DAY):
    """
    Analyzes the dataset to compute expected vs actual readings per sensor/inverter.
    """
    inverter_count = df['SOURCE_KEY'].nunique()
    expected_readings = days * readings_per_day * inverter_count
    actual_readings = len(df)
    diff = actual_readings - expected_readings

    print(f"\n--- {plant_label} Generation Data ---")
    print(f"1. Unique Inverters identified: {inverter_count}")
    print(f"2. Expected Readings (Target):  {expected_readings:,}")
    print(f"   (Calculation: {days} days * {readings_per_day} readings * {inverter_count} inverters)")
    print(f"3. Actual Readings (Raw):       {actual_readings:,}")
    print(f"4. Conclusion:                  {diff:+,} Rows")


p1_gen_stats = analyse_dataset(plant1_gen, "Plant 1")
p2_gen_stats = analyse_dataset(plant2_gen, "Plant 2")
p1_weather_stats = analyse_dataset(plant1_weather, "Plant 1 Weather")
p2_weather_stats = analyse_dataset(plant2_weather, "Plant 2 Weather")


--- Plant 1 Generation Data ---
1. Unique Inverters identified: 22
2. Expected Readings (Target):  71,808
   (Calculation: 34 days * 96 readings * 22 inverters)
3. Actual Readings (Raw):       1,021,186
4. Conclusion:                  +949,378 Rows

--- Plant 2 Generation Data ---
1. Unique Inverters identified: 22
2. Expected Readings (Target):  71,808
   (Calculation: 34 days * 96 readings * 22 inverters)
3. Actual Readings (Raw):       1,421,196
4. Conclusion:                  +1,349,388 Rows

--- Plant 1 Weather Generation Data ---
1. Unique Inverters identified: 1
2. Expected Readings (Target):  3,264
   (Calculation: 34 days * 96 readings * 1 inverters)
3. Actual Readings (Raw):       3,182
4. Conclusion:                  -82 Rows

--- Plant 2 Weather Generation Data ---
1. Unique Inverters identified: 1
2. Expected Readings (Target):  3,264
   (Calculation: 34 days * 96 readings * 1 inverters)
3. Actual Readings (Raw):       3,259
4. Conclusion:                  -5 Rows


**Conclusion:**
- When slicing data, we observe exact time stamps with all variables equal except for "operating_condition". Understanding this in detail and cleaning accordingly will be critical for the "Classification of Operating Conditions" task.

In [4]:
## Check data types
def get_types_report(df):
    """Generate a report of pandas dtypes and unique Python types for each column in the DataFrame."""
    types = {}
    for col in df.columns:
        # Get unique python types in the column (useful for spotting mixed types)
        py_types = df[col].map(lambda x: type(x).__name__).unique().tolist()
        types[col] = py_types

    result_df = pd.DataFrame({
        'column': list(types.keys()),
        'pandas_dtype': [df[col].dtype for col in types.keys()],
        'python_types': [types[col] for col in types.keys()]
    })
    return result_df

# Get unique Python types and pandas dtypes for the four datasets
print("Plant 1 Generation - Data Types Report:")
print(get_types_report(plant1_gen), "\n")

print("Plant 1 Weather - Data Types Report:")
print(get_types_report(plant1_weather), "\n")

print("Plant 2 Generation - Data Types Report:")
print(get_types_report(plant2_gen), "\n")

print("Plant 2 Weather - Data Types Report:")
print(get_types_report(plant2_weather))

Plant 1 Generation - Data Types Report:
                column pandas_dtype  python_types
0            DATE_TIME       object         [str]
1             PLANT_ID        int64         [int]
2           SOURCE_KEY       object         [str]
3             DC_POWER      float64       [float]
4             AC_POWER      float64       [float]
5          DAILY_YIELD      float64       [float]
6          TOTAL_YIELD      float64       [float]
7                  day        int64         [int]
8  Operating_Condition       object  [str, float] 

Plant 1 Weather - Data Types Report:
                column pandas_dtype python_types
0            DATE_TIME       object        [str]
1             PLANT_ID        int64        [int]
2           SOURCE_KEY       object        [str]
3  AMBIENT_TEMPERATURE      float64      [float]
4   MODULE_TEMPERATURE      float64      [float]
5          IRRADIATION      float64      [float] 

Plant 2 Generation - Data Types Report:
                column pandas_dtype 

**Conclusion:**
- Mixed formats: In "Plant 1 - Generation data" the variable "Operating_Condition" mariable exhibits mixed data types, strings and float. This means there are probably missing values in this column (NaN)

- Integers for categorical information: Variable Plant_ID is currently encoded as an integer across, is shown as an integer. However, it serves as a unique categorical identifier rather than a quantitative metric. This distinction must be explicitly noted during modeling to ensure algorithms do not misinterpret the ID as having numerical magnitude or order

#### B. Missing Values

In [5]:
## Check for missing values in the four data sets

def get_missing_data_report(df):
    """Generate a report of missing data percentages for each column in the DataFrame."""
    missing_data_report = pd.DataFrame({
        'Columns': df.columns,
        'Missing Values': df.isna().sum().values,
        'Percentage Missing': ((df.isna().sum().values / len(df)) * 100).round(2)
    })
    return missing_data_report

# Generate and print missing data report for the datasets
print("Power Generation 1 - Missing Values Report:")
print(get_missing_data_report(plant1_gen),"\n")
print("Weather Sensor 1 - Missing Values Report:")
print(get_missing_data_report(plant1_weather))
print("Power Generation 2 - Missing Values Report:")
print(get_missing_data_report(plant2_gen),"\n")
print("Weather Sensor 2- Missing Values Report:")
print(get_missing_data_report(plant2_weather))

Power Generation 1 - Missing Values Report:
               Columns  Missing Values  Percentage Missing
0            DATE_TIME               0                0.00
1             PLANT_ID               0                0.00
2           SOURCE_KEY               0                0.00
3             DC_POWER               0                0.00
4             AC_POWER               0                0.00
5          DAILY_YIELD               0                0.00
6          TOTAL_YIELD               0                0.00
7                  day               0                0.00
8  Operating_Condition           23098                2.26 

Weather Sensor 1 - Missing Values Report:
               Columns  Missing Values  Percentage Missing
0            DATE_TIME               0                 0.0
1             PLANT_ID               0                 0.0
2           SOURCE_KEY               0                 0.0
3  AMBIENT_TEMPERATURE               0                 0.0
4   MODULE_TEMPERATURE     

In [6]:
## Check for missing data ranges (rows) in the four data sets
# -------------------------------------------------------
# Generate a report
# -------------------------------------------------------
def get_time_gap_report(df):
    """
    Generate a report of missing time intervals in the DataFrame.
    Assumes 'DATE_TIME' column exists.
    """
    # Ensure datetime and sort
    df = df.copy()
    df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'])
    df = df.sort_values('DATE_TIME')
    
    # Create the expected grid for the intervals (perfect 15 min intervals)
    start = df['DATE_TIME'].min()
    end = df['DATE_TIME'].max()
    expected_range = pd.date_range(start=start, end=end, freq='15min')
    
    # Calculate statistics
    expected_count = len(expected_range)
    actual_count = df['DATE_TIME'].nunique()
    missing_count = expected_count - actual_count
    pct_missing = (missing_count / expected_count) * 100
    
    # Create Report DataFrame
    report = pd.DataFrame({
        'Metric': ['Start Time', 'End Time', 'Expected Intervals', 'Actual Intervals', 'Missing Intervals', '% Missing'],
        'Value': [start, end, expected_count, actual_count, missing_count, round(pct_missing, 2)]
    })
    return report

# Generate and print time gap reports
print("Power Generation 1 - Time Gap Report:")
print(get_time_gap_report(plant1_gen),"\n")

print("Weather Sensor 1 - Time Gap Report:")
print(get_time_gap_report(plant1_weather),"\n")

print("Power Generation 2 - Time Gap Report:")
print(get_time_gap_report(plant2_gen),"\n")

print("Weather Sensor 2 - Time Gap Report:")
print(get_time_gap_report(plant2_weather))

Power Generation 1 - Time Gap Report:
               Metric                Value
0          Start Time  2020-01-06 00:00:00
1            End Time  2020-12-06 23:45:00
2  Expected Intervals                32256
3    Actual Intervals                 3158
4   Missing Intervals                29098
5           % Missing                90.21 

Weather Sensor 1 - Time Gap Report:
               Metric                Value
0          Start Time  2020-05-15 00:00:00
1            End Time  2020-06-17 23:45:00
2  Expected Intervals                 3264
3    Actual Intervals                 3182
4   Missing Intervals                   82
5           % Missing                 2.51 

Power Generation 2 - Time Gap Report:
               Metric                Value
0          Start Time  2020-05-15 00:00:00
1            End Time  2020-06-17 23:45:00
2  Expected Intervals                 3264
3    Actual Intervals                 3259
4   Missing Intervals                    5
5           % Missing   

**Conclusion:** 

Missing values (NaN):
- Plant 1: Generation data exhibits missing values. There are about 2.3% missing values in the Operating_Condition column of the data set. On the contrary, weather data shows 0 missing values (empty cells)
- Plant 2: There are no missing values in the generation or weather data.

Missing temporal datetimes (entire rows):
- Plant 1: Is the only one with a time range different from the other datasets. We suspect the date format is not consistent and it is also the reason why the missing value count is high (around 90%) thus needs to be fixed.
- Plant 2: The datetime coverage is nearly perfect, it only misses 5 intervals (0.15%) of information.

#### C. Inconsistencies & Anomalies

##### Date Format Fixes

In [7]:
# Ensure all time columns are datetime objects
plant1_weather['DATE_TIME'] = pd.to_datetime(plant1_weather['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')
plant2_weather['DATE_TIME'] = pd.to_datetime(plant2_weather['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')
plant2_gen['DATE_TIME'] = pd.to_datetime(plant2_gen['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')

In [8]:
# The timerange of the data is known to be within May 15 and June 17
print("Data timerange:", plant1_weather['DATE_TIME'].min(), "to", plant1_weather['DATE_TIME'].max())

Data timerange: 2020-05-15 00:00:00 to 2020-06-17 23:45:00


In [9]:
# Resolve DATE_TIME format ambiguity in Plant 1 Generation Data
# Extract month and day parts from DATE_TIME
plant1_gen = plant1_gen.join(plant1_gen['DATE_TIME'].str.extract(r'-(\d{2})-(\d{2}) ')).rename(columns={0: 'mid', 1: 'right'})

# Determine date format based on extracted parts knowing that the timerange of the data is known to be within May 15 and June 17
plant1_gen['date_format'] = ''
plant1_gen.loc[(plant1_gen['mid']=='05') & (plant1_gen['right']=='06'), "date_format"] = 'dd-mm'
plant1_gen.loc[(plant1_gen['mid']=='06') & (plant1_gen['right']=='05'), "date_format"] = 'mm-dd'
plant1_gen.loc[((plant1_gen['mid']=='05') | (plant1_gen['mid']=='06')) & (plant1_gen['date_format'] == ''), "date_format"] = 'mm-dd'
plant1_gen.loc[((plant1_gen['right']=='05') | (plant1_gen['right']=='06')) & (plant1_gen['date_format'] == ''), "date_format"] = 'dd-mm'

# Convert DATE_TIME to datetime using the determined format
mask_ddmm = plant1_gen['date_format'] == 'dd-mm'
mask_mmdd = plant1_gen['date_format'] == 'mm-dd'
plant1_gen.loc[mask_ddmm,"DATE_TIME"] = pd.to_datetime(plant1_gen.loc[mask_ddmm,"DATE_TIME"], format='%Y-%d-%m %H:%M:%S')
plant1_gen.loc[mask_mmdd,"DATE_TIME"] = pd.to_datetime(plant1_gen.loc[mask_mmdd,"DATE_TIME"], format='%Y-%m-%d %H:%M:%S')
plant1_gen['DATE_TIME'] = pd.to_datetime(plant1_gen['DATE_TIME']) # Final conversion to datetime
plant1_gen = plant1_gen.drop(columns=['mid', 'right', 'date_format'])

In [10]:
# Generate and print the time gap report again for Plant 1 Generation Data (after fixing DATE_TIME format)
print("Power Generation 1 - Time Gap Report:")
print(get_time_gap_report(plant1_gen),"\n")

Power Generation 1 - Time Gap Report:
               Metric                Value
0          Start Time  2020-05-15 00:00:00
1            End Time  2020-06-17 23:45:00
2  Expected Intervals                 3264
3    Actual Intervals                 3158
4   Missing Intervals                  106
5           % Missing                 3.25 



With the right date format correction, plant generation 1 presents a 3.25% of missing date intervals (instead of 90%+ before the fix).

##### Power Format Fixes

In [11]:
print("Power Generation Plant 1\n", plant1_gen[['DC_POWER','AC_POWER']].describe(),"\n")
print("Power Generation Plant 2\n", plant2_gen[['DC_POWER','AC_POWER']].describe())

Power Generation Plant 1
            DC_POWER      AC_POWER
count  1.021186e+06  1.021186e+06
mean   3.214259e+03  3.143240e+02
std    4.083181e+03  3.989361e+02
min    0.000000e+00  0.000000e+00
25%    0.000000e+00  0.000000e+00
50%    5.161250e+02  4.992500e+01
75%    6.504625e+03  6.371375e+02
max    1.447112e+04  1.410950e+03 

Power Generation Plant 2
            DC_POWER      AC_POWER
count  1.421196e+06  1.421196e+06
mean   2.446906e+02  2.393180e+02
std    3.675888e+02  3.592099e+02
min    0.000000e+00  0.000000e+00
25%    0.000000e+00  0.000000e+00
50%    0.000000e+00  0.000000e+00
75%    4.424643e+02  4.340333e+02
max    1.420933e+03  1.385420e+03


From the observation of the DC and AC power generation descriptive statistics, we notice that Plant 1's DC power values are an order of magnitude higher than Plant 2's. To fix this error in the data, we scale down Plant 1's DC power values by a factor of 10 to ensure consistency across both plants.

In [12]:
# Fix DC_POWER scale issue in Plant 1 Generation Data
plant1_gen['DC_POWER'] = plant1_gen['DC_POWER'] / 10

In [13]:
# Reprint descriptive statistics after fixing DC_POWER scale issue
print("Power Generation Plant 1\n", plant1_gen[['DC_POWER','AC_POWER']].describe(),"\n")
print("Power Generation Plant 2\n", plant2_gen[['DC_POWER','AC_POWER']].describe())

Power Generation Plant 1
            DC_POWER      AC_POWER
count  1.021186e+06  1.021186e+06
mean   3.214259e+02  3.143240e+02
std    4.083181e+02  3.989361e+02
min    0.000000e+00  0.000000e+00
25%    0.000000e+00  0.000000e+00
50%    5.161250e+01  4.992500e+01
75%    6.504625e+02  6.371375e+02
max    1.447112e+03  1.410950e+03 

Power Generation Plant 2
            DC_POWER      AC_POWER
count  1.421196e+06  1.421196e+06
mean   2.446906e+02  2.393180e+02
std    3.675888e+02  3.592099e+02
min    0.000000e+00  0.000000e+00
25%    0.000000e+00  0.000000e+00
50%    0.000000e+00  0.000000e+00
75%    4.424643e+02  4.340333e+02
max    1.420933e+03  1.385420e+03


Now the power values look consistent between both plants.

##### Further Checks

In [14]:
## Inconsistencies are checked for different situations for all data sets
# Grouping for iteration
gen_data = [("Plant 1", plant1_gen), ("Plant 2", plant2_gen)]
weather_data = [("Plant 1", plant1_weather), ("Plant 2", plant2_weather)]


## Part A: Generation data
print("\n--- POWER GENERATION CHECKS ---")

# CHECK 1: Efficiency violation 
# Impossible for AC Output > DC Input (inverter efficiency can't be > 100%)

print("\n[Gen Check 1] Efficiency violation (AC > DC)")
for name, df in gen_data:
    # We allow a tiny buffer (0.1 kW) for sensor timing mismatch
    errors = df[df['AC_POWER'] > df['DC_POWER'] + 0.1]
    print(f"  {name}: {len(errors)} rows failed.")

# CHECK 2: Negative power
# Solar panels cannot consume power (cannot be negative)
print("\n[Gen Check 2] Negative power (AC or DC < 0)")
for name, df in gen_data:
    errors = df[(df['AC_POWER'] < 0) | (df['DC_POWER'] < 0)]
    print(f"  {name}: {len(errors)} rows failed.")


## Part B: Weather data

print("\n--- WEATHER CHECKS ---")

# CHECK 1: No irradiance at night
# Irradiance > 0 when it is dark (10 PM - 4 AM short range assuming its summer)
print("\n[Weather Check 1] Night irradiance (Irradiation > 0 at Night)")
for name, df in weather_data:
    hour = df['DATE_TIME'].dt.hour
    night_mask = (hour >= 22) | (hour < 4)
    errors = df[night_mask & (df['IRRADIATION'] > 0)]
    print(f"  {name}: {len(errors)} rows failed.")

# CHECK 2: Panel being hot without irradiance at night
# No Sun (0 Irr) but Module is significantly hotter than Ambient
print("\n[Weather Check 2] Hot panel at night (Mod Temp > Amb Temp + 5°C w/ no Sun)")
for name, df in weather_data:
    # If Irradiance is 0, Module shouldn't be hot compared to air
    errors = df[(df['IRRADIATION'] == 0) & 
                (df['MODULE_TEMPERATURE'] > df['AMBIENT_TEMPERATURE'] + 5.0)]
    print(f"  {name}: {len(errors)} rows failed.")

# CHECK 3: Broken sensors
# Ambient Temp stays exactly the same for 4 consecutive readings (1 hour). Testing for a extreme case.
print("\n[Weather Check 3] Broken sensors (Ambient Temp frozen for 1hr)")
for name, df in weather_data:
    df_sorted = df.sort_values(by='DATE_TIME') 
    
    # True if current value equals previous value
    is_frozen = df_sorted['AMBIENT_TEMPERATURE'].diff() == 0
    
    # Rolling sum: if 4 consecutive rows are "True", we have a 1-hour freeze
    frozen_blocks = is_frozen.rolling(4).sum() == 4
    print(f"  {name}: {frozen_blocks.sum()} frozen sequences found.")


--- POWER GENERATION CHECKS ---

[Gen Check 1] Efficiency violation (AC > DC)
  Plant 1: 396 rows failed.
  Plant 2: 396 rows failed.

[Gen Check 2] Negative power (AC or DC < 0)
  Plant 1: 0 rows failed.
  Plant 2: 0 rows failed.

--- WEATHER CHECKS ---

[Weather Check 1] Night irradiance (Irradiation > 0 at Night)
  Plant 1: 0 rows failed.
  Plant 2: 24 rows failed.

[Weather Check 2] Hot panel at night (Mod Temp > Amb Temp + 5°C w/ no Sun)
  Plant 1: 0 rows failed.
  Plant 2: 0 rows failed.

[Weather Check 3] Broken sensors (Ambient Temp frozen for 1hr)
  Plant 1: 0 frozen sequences found.
  Plant 2: 0 frozen sequences found.


**Conclusion:** 

Generation data
- Plant 1: There are 396 instances (~0.6%) of efficiency violations (system reports that AC output is higher than DC, which is physically imposible).
- Plant 2: There are 396 instances (~0.6%) of efficiency violations (system reports that AC output is higher than DC, which is physically imposible).

Weather data
- Plant 1: Passed all consistency checks
- Plant 2: 24 instances (~0.7%) of "Night Sun" (Irradiance > 0 at night) This culd be due to the sunrise/sunset exact timings or callibration

### 2.2.2 Data Handling

#### A. Fixing Operating Condition Column

##### Problem Identification (using Plant 1)

The problem of the Operating Condition column is that at the time of its creation, instead of assigning each category (Optimal / Suboptimal) to the respective source key (by joining them), the whole set of operating conditions was assigned to each source key for each timestamp, resulting in duplicated rows with different operating conditions for the same source key and timestamp (that is, by broadcast).

In [15]:
# Get the sequence of operating conditions per datetime, plant, and source key
operating_condition_per_datetime = plant2_gen.groupby(['DATE_TIME','PLANT_ID','SOURCE_KEY'],as_index=False)['Operating_Condition'].apply(list)
print(operating_condition_per_datetime.tail().to_markdown())
# Create a sequence string for easier comparison
operating_condition_per_datetime['operating_condition_sequence'] = operating_condition_per_datetime['Operating_Condition'].apply(lambda x: str('-'.join(x)))
# Check the number of unique operating condition sequences per datetime
unique_operating_condition_per_datetime = operating_condition_per_datetime.groupby(['DATE_TIME'],as_index=False)['operating_condition_sequence'].nunique()
# Check the amount of unique sequences of operating conditions per datetime assigned to each duplicated source key
unique_operating_condition_per_datetime['operating_condition_sequence'].unique()

|       | DATE_TIME           |   PLANT_ID | SOURCE_KEY      | Operating_Condition                                                                                                                                                                                                                                                                                            |
|------:|:--------------------|-----------:|:----------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 67693 | 2020-06-17 23:45:00 |    4136001 | q49J1IKaHRwDQnt | ['Suboptimal', 'Suboptimal', 'Optimal', 'Suboptimal', 'Suboptimal', 'Suboptimal', 'Suboptimal', 'Suboptimal', 'Suboptimal', 'Suboptimal', 'Suboptimal', 'Suboptimal', 'Suboptimal', 'Suboptimal', 'Subopt

array([1])

This confirms the assignation problem of the Operating Condition column comented earlier, since there is always only one unique sequence of operating conditions per datetime suggesting that the operating condition data was wrongly broadcasted to all source keys instead of being joined properly.

##### Problem Solution

In [16]:
def correct_operating_conditions(df: pd.DataFrame, source_key_col: str = 'SOURCE_KEY', plant_id_col: str = 'PLANT_ID') -> pd.DataFrame:
    """
    Corrects the Operating_Condition column by resolving the broadcast error.

    The function assumes that for a given DATE_TIME, the true sequence of 
    Operating_Conditions was inadvertently broadcast to every row, and this 
    sequence must be mapped element-wise to the alphabetically sorted list of 
    SOURCE_KEYs.

    Args:
        df (pd.DataFrame): The input DataFrame containing duplicated operating conditions on source keys.
        source_key_col (str): The name of the column identifying the inverter (e.g., 'SOURCE_KEY').
        plant_id_col (str): The name of the column identifying the plant (e.g., 'PLANT_ID').

    Returns:
        pd.DataFrame: A new DataFrame with one row per unique measurement and 
                      the corrected 'Operating_Condition' column.
    """
    
    # 1. Prepare Data for Mapping
    # Get the unique Operating_Condition sequence for each DATE_TIME.
    # We group by the full ID set, take the first condition list, then aggregate by DATE_TIME/PLANT_ID.
    condition_list_per_datetime = (
        df.groupby(['DATE_TIME', plant_id_col, source_key_col], as_index=False)['Operating_Condition']
        .apply(list)
    )
    # The 'first()' aggregates the lists by DATE_TIME/PLANT_ID, relying on the fact that
    # the list is identical for all source keys in that time frame.
    condition_list_per_datetime = condition_list_per_datetime.groupby(
        ['DATE_TIME', plant_id_col], as_index=False
    )['Operating_Condition'].first()

    # Get the unique and sorted list of SOURCE_KEYs since they may not all be present in all time frames
    source_keys_per_datetime = df.groupby(['DATE_TIME', plant_id_col], as_index=False).agg(
        {source_key_col: lambda x: sorted(x.unique())} # Ensures SOURCE_KEYs are always alphabetically sorted
    )

    # Merge the two to get the final mapping table
    mapping_df = source_keys_per_datetime.merge(
        condition_list_per_datetime, on=['DATE_TIME', plant_id_col]
    )

    # 2. Create the Source Key -> Condition Mapping Dictionary
    def create_condition_mapping(row):
        source_keys = row[source_key_col]
        operating_conditions = row['Operating_Condition']
        mapping = {}
        
        # The core logic: map the alphabetically sorted keys to the extracted sequence.
        for i, key in enumerate(source_keys):
            if i < len(operating_conditions):
                mapping[key] = operating_conditions[i]
            else:
                # Assign NaN for missing operating conditions (critical for the known data gap)
                mapping[key] = np.nan 
        return mapping

    mapping_df['source_key_to_condition_map'] = mapping_df.apply(create_condition_mapping, axis=1)

    # 3. Apply Correction and Deduplicate
    # Create the base DataFrame containing unique measurements (without the redundant condition column)
    cols_to_drop = ['Operating_Condition']
    df_base = df.drop_duplicates(subset=['DATE_TIME', plant_id_col, source_key_col]).copy()
    df_base = df_base.drop(columns=cols_to_drop)
    
    # 4. Assign the Corrected Operating Condition
    # Use the mapping_df to look up the correct condition for each unique row in df_base
    df_base['Operating_Condition'] = df_base.apply(
        lambda row: mapping_df[
            (mapping_df['DATE_TIME'] == row['DATE_TIME']) &
            (mapping_df[plant_id_col] == row[plant_id_col])
        ]['source_key_to_condition_map'].values[0].get(row[source_key_col], np.nan),
        axis=1
    )
    
    return df_base

In [17]:
plant1_gen_fixed = correct_operating_conditions(plant1_gen)
plant2_gen_fixed = correct_operating_conditions(plant2_gen)

#### B. Generating Missing Timestamps

In [18]:
from typing import List

def impute_power_by_mean_interpolation(
    df: pd.DataFrame,
    power_cols: List[str] = ['AC_POWER', 'DC_POWER'],
    source_key_col: str = 'SOURCE_KEY',
    time_col: str = 'DATE_TIME',
    freq: str = '15min'
) -> pd.DataFrame:
    """
    Densifies the DataFrame by filling missing (DATE_TIME, SOURCE_KEY) combinations
    and then imputes missing power values using the interpolated mean of all inverters 
    for that specific time step.

    This solves two problems: missing time steps (Type 1) and missing inverters (Type 2).

    Args:
        df (pd.DataFrame): The input DataFrame (e.g., plant1_gen, plant2_gen).
        power_cols (List[str]): List of columns to impute.
        source_key_col (str): Column identifying the inverter ('SOURCE_KEY').
        time_col (str): Column identifying the timestamp ('DATE_TIME').
        freq (str): The time series frequency (e.g., '15min').

    Returns:
        pd.DataFrame: A dense DataFrame with imputed AC/DC Power.
    """
    
    df = df.copy()
    
    # Ensure DATE_TIME is a proper datetime object
    df[time_col] = pd.to_datetime(df[time_col])

    # 1. Create a dense MultiIndex (All combinations of Time and Source Key)
    full_timerange = pd.date_range(
        start=df[time_col].min(), 
        end=df[time_col].max(), 
        freq=freq
    )
    all_source_keys = df[source_key_col].unique()

    full_index = pd.MultiIndex.from_product(
        [full_timerange, all_source_keys], 
        names=[time_col, source_key_col],
    )

    # Reindex the original DataFrame onto the full grid (introduces NaNs for missing rows)
    df = (
        df.set_index([time_col, source_key_col])
        .reindex(full_index)
        .reset_index()
    )

    # 2. Calculate the MEAN Power across available SOURCE_KEYs for each DATE_TIME.
    plant_mean_power = df.groupby(time_col)[power_cols].mean()

    # 3. Interpolate the calculated MEANS (fills Type 1 missing steps).
    # This fills gaps where ALL inverters were missing data.
    plant_mean_power[power_cols] = plant_mean_power[power_cols].interpolate(method='time', limit_direction='both')

    # 4. Rename columns for clarity
    mean_cols = {col: f'mean_{col}_interpolated' for col in power_cols}
    plant_mean_power = plant_mean_power.rename(columns=mean_cols)
    
    # 5. Merge the interpolated mean values back into the main DataFrame.
    df = (
        df.set_index(time_col)
        .join(plant_mean_power, how='left')
        .reset_index()
    )

    # 6. Impute AC/DC Power NaNs using the corresponding mean value (fills Type 2 missing inverters).
    for original_col, mean_col in mean_cols.items():
        df[original_col] = df[original_col].fillna(df[mean_col])

    # 7. Final Cleanup (Remove intermediate columns)
    df = df.drop(columns=mean_cols.values())

    df['PLANT_ID'] = df['PLANT_ID'].fillna(df['PLANT_ID'].mode()[0])

    return df


plant1_cleaned = impute_power_by_mean_interpolation(
    df=plant1_gen_fixed, 
    power_cols=['AC_POWER', 'DC_POWER'], 
    source_key_col='SOURCE_KEY',
    time_col='DATE_TIME',
    freq='15min'
)

plant2_cleaned = impute_power_by_mean_interpolation(
    df=plant2_gen_fixed, 
    power_cols=['AC_POWER', 'DC_POWER'], 
    source_key_col='SOURCE_KEY',
    time_col='DATE_TIME',
    freq='15min'
)

In [20]:
def recalculate_yields(df):
    df = df.copy()

    # day/month for per-day grouping
    df['day'] = df['DATE_TIME'].dt.day
    df['month'] = df['DATE_TIME'].dt.month

    # DAILY_YIELD: cumulative AC_POWER per SOURCE_KEY per day
    df['DAILY_YIELD'] = df.groupby(['SOURCE_KEY', 'month', 'day'])['AC_POWER'].cumsum()

    # ensure TOTAL_YIELD column exists for the earliest-yield lookup
    if 'TOTAL_YIELD' not in df.columns:
        df['TOTAL_YIELD'] = np.nan

    # get earliest DATE_TIME per SOURCE_KEY and corresponding TOTAL_YIELD
    min_dates = df.groupby('SOURCE_KEY', as_index=False)['DATE_TIME'].min()
    first_yield = (min_dates
                   .merge(df[['SOURCE_KEY', 'DATE_TIME', 'TOTAL_YIELD']],
                          on=['SOURCE_KEY', 'DATE_TIME'],
                          how='left')
                   [['SOURCE_KEY', 'DATE_TIME', 'TOTAL_YIELD']]
                   .drop_duplicates('SOURCE_KEY')
                   .rename(columns={'DATE_TIME': 'FIRST_DATE_TIME', 'TOTAL_YIELD': 'FIRST_TOTAL_YIELD'}))

    # merge FIRST_TOTAL_YIELD and fill missing with 0
    df = df.merge(first_yield[['SOURCE_KEY', 'FIRST_TOTAL_YIELD']], on='SOURCE_KEY', how='left')
    df['FIRST_TOTAL_YIELD'] = df['FIRST_TOTAL_YIELD'].fillna(0.0)

    # recompute TOTAL_YIELD
    df['TOTAL_YIELD'] = df['DAILY_YIELD'] + df['FIRST_TOTAL_YIELD']

    # cleanup helper columns
    df.drop(columns=['day', 'month', 'FIRST_TOTAL_YIELD'], inplace=True)
    return df

# Apply to both plants and assign back
plant1_cleaned = recalculate_yields(plant1_cleaned)
plant2_cleaned = recalculate_yields(plant2_cleaned)

# When missing datetime data, yield calculations are not accurate.

#### C. Addressing Outliers

#####  Outliers Detection

In [ ]:
# -------------------------------------------------------
# Power Generation Outlier Detection
# -------------------------------------------------------

# Selected columns to analyze
ANALYZE_COLS = ['DC_POWER', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD']

#Function to detect outliers - Using IQR method
def detect_outliers(data, name, cols):
    """Detect outliers using IQR method."""
    print(f"\n{name}")
    print("-" * 60)
    
    for col in cols:
        Q1, Q3 = data[col].quantile([0.25, 0.75])
        IQR = Q3 - Q1
        lower = max(0, Q1 - 1.5 * IQR)
        upper = Q3 + 1.5 * IQR
        
        outliers = ((data[col] < lower) | (data[col] > upper)).sum()
        outlier_pct = (outliers / len(data)) * 100
        
        print(f"{col:20} | Outliers: {outliers:5} ({outlier_pct:5.2f}%) | Bounds: [{lower:8.2f}, {upper:8.2f}]")

print("=" * 80)
print("BATCH 1: GENERATION DATA - OUTLIER DETECTION")
print("=" * 80)

detect_outliers(plant1_cleaned, "Plant 1 Generation", ANALYZE_COLS)
detect_outliers(plant2_cleaned, "Plant 2 Generation", ANALYZE_COLS)

BATCH 1: GENERATION DATA - OUTLIER DETECTION

Plant 1 Generation
------------------------------------------------------------
DC_POWER             | Outliers:     0 ( 0.00%) | Bounds: [    0.00,  1552.55]
AC_POWER             | Outliers:     0 ( 0.00%) | Bounds: [    0.00,  1520.90]
DAILY_YIELD          | Outliers:     0 ( 0.00%) | Bounds: [    0.00, 65402.66]
TOTAL_YIELD          | Outliers:  3264 ( 4.55%) | Bounds: [5112336.44, 8357473.74]

Plant 2 Generation
------------------------------------------------------------
DC_POWER             | Outliers:  2623 ( 3.65%) | Bounds: [    0.00,  1148.66]
AC_POWER             | Outliers:  2539 ( 3.54%) | Bounds: [    0.00,  1126.54]
DAILY_YIELD          | Outliers:     0 ( 0.00%) | Bounds: [    0.00, 49991.86]
TOTAL_YIELD          | Outliers:     0 ( 0.00%) | Bounds: [    0.00, 3340967183.32]


In [33]:
# -------------------------------------------------------
# Power Generation Outlier Detection
# -------------------------------------------------------

ANALYZE_COLS = ['IRRADIATION', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE']

print("\n" + "=" * 80)
print("BATCH 2: WEATHER DATA - OUTLIER DETECTION")
print("=" * 80)

detect_outliers(plant1_weather, "Plant 1 Weather", ANALYZE_COLS)
detect_outliers(plant2_weather, "Plant 2 Weather", ANALYZE_COLS)


BATCH 2: WEATHER DATA - OUTLIER DETECTION

Plant 1 Weather
------------------------------------------------------------
IRRADIATION          | Outliers:     2 ( 0.06%) | Bounds: [    0.00,     1.12]
AMBIENT_TEMPERATURE  | Outliers:     0 ( 0.00%) | Bounds: [   14.88,    35.74]
MODULE_TEMPERATURE   | Outliers:     0 ( 0.00%) | Bounds: [    0.00,    71.63]

Plant 2 Weather
------------------------------------------------------------
IRRADIATION          | Outliers:     1 ( 0.03%) | Bounds: [    0.00,     1.10]
AMBIENT_TEMPERATURE  | Outliers:     0 ( 0.00%) | Bounds: [   14.92,    40.74]
MODULE_TEMPERATURE   | Outliers:     2 ( 0.06%) | Bounds: [    0.00,    65.63]


##### Outlier Capping

In [34]:
# -------------------------------------------------------
# Power Generation Outlier Capping
# -------------------------------------------------------

CAP_COLS = ['DC_POWER', 'AC_POWER']

def cap_outliers_inplace(data, name, cols):
    """Cap outliers using IQR method - replaces values in original columns."""
    print(f"\n{name}")
    
    for col in cols:
        Q1, Q3 = data[col].quantile([0.25, 0.75])
        IQR = Q3 - Q1
        lower = max(0, Q1 - 1.5 * IQR)
        upper = Q3 + 1.5 * IQR
        
        # Replace values in-place
        data[col] = data[col].clip(lower=lower, upper=upper)
    
    print(f"✓ Capped in-place: {', '.join(cols)}")

print("\n" + "=" * 80)
print("BATCH 1: GENERATION DATA - APPLYING CAPPING")
print("=" * 80)

cap_outliers_inplace(plant1_cleaned, "Plant 1 Generation", CAP_COLS)
cap_outliers_inplace(plant2_cleaned, "Plant 2 Generation", CAP_COLS)


BATCH 1: GENERATION DATA - APPLYING CAPPING

Plant 1 Generation
✓ Capped in-place: DC_POWER, AC_POWER

Plant 2 Generation
✓ Capped in-place: DC_POWER, AC_POWER


In [35]:
# -------------------------------------------------------
# Power Generation Outlier Capping
# -------------------------------------------------------

CAP_COLS = ['IRRADIATION', 'AMBIENT_TEMPERATURE']

print("\n" + "=" * 80)
print("BATCH 2: WEATHER DATA - APPLYING CAPPING")
print("=" * 80)

cap_outliers_inplace(plant1_weather, "Plant 1 Weather", CAP_COLS)
cap_outliers_inplace(plant2_weather, "Plant 2 Weather", CAP_COLS)


BATCH 2: WEATHER DATA - APPLYING CAPPING

Plant 1 Weather
✓ Capped in-place: IRRADIATION, AMBIENT_TEMPERATURE

Plant 2 Weather
✓ Capped in-place: IRRADIATION, AMBIENT_TEMPERATURE


### 2.2.3 Integration

In [ ]:
plants_gen = pd.concat([plant1_cleaned, plant2_cleaned])
plants_weather = pd.concat([plant1_weather, plant2_weather])

In [ ]:
plants_df = plants_gen.merge(plants_weather[plants_weather.columns.difference(['SOURCE_KEY'])], on=['DATE_TIME', 'PLANT_ID'], how='inner')

In [ ]:
plants_df.to_csv('../data/processed/plants_combined_data.csv', index=False)

In [ ]:
print("Percentage of data retained after merging generation and weather data: ",
      round(len(plants_df)/(len(plant1_gen)+len(plant2_gen))*100, 3), "%")
print("Lost data rows:", (len(plant1_gen)+len(plant2_gen)) - len(plants_df))

## 2.3 Exploratory Data Analysis

### 2.3.1 Statistical Summary

In [ ]:
plants_df.describe()

### 2.3.2 Visualizations

In [ ]:
from matplotlib import rcParams
import matplotlib.pyplot as plt
import seaborn as sns

darkblue = (60/255, 85/255, 125/255)
ddarkblue = (35/255, 51/255, 81/255)
darkcyan = (90/255, 160/255, 190/255)
ddarkcyan = (30/255, 100/255, 130/255)
background = (234/255, 234/255, 241/255)
error_color = (236/255, 175/255, 74/255)
lightblue_almost_white = (245/255, 253/255, 255/255)  # pale blue near white

color_palette = [
    (17/255, 138/255, 178/255),  # Deep cyan
    (6/255, 214/255, 160/255),   # Bright teal
    (136/255, 84/255, 208/255),  # Pastel purple
    (239/255, 71/255, 111/255),  # Vibrant pink
    (255/255, 180/255, 51/255),  # Darkened yellow
    (7/255, 59/255, 76/255)      # Dark slate
]

def set_style():
    # Plot
    sns.set_theme(style="darkgrid")

    # Enable LaTeX fonts
    rcParams.update({
        "text.usetex": True,
        "font.family": "serif",
        "text.latex.preamble": r"\usepackage{mathptmx}",
        "axes.unicode_minus": False
    })

set_style()

#### A. Power Generation Data Plots

##### A.1 Plant Comparison of AC Power Output (2 plots)

In [ ]:
# Histogram of AC Power Output for Plants 1 and 2 (excluding zero values)
# -------------------------------------------------------
fig, ax = plt.subplots(figsize=(10, 6))

sns.histplot(
    data=plant1_gen[plant1_gen['AC_POWER'] > 0],
    x="AC_POWER",
    bins=70,
    color=darkblue,
    kde=False,
    stat='frequency',
    label='Plant 1',
    edgecolor='black',
    alpha=0.7,
    linewidth=0.5
)

sns.histplot(
    data=plant2_gen[plant2_gen['AC_POWER'] > 0],
    x="AC_POWER",
    bins=70,
    color=darkcyan,
    kde=False,
    stat='frequency',
    label='Plant 2',
    edgecolor='black',
    alpha=0.7,
    linewidth=0.5
)

# Axis limits
# plt.xlim(left=0, right=1000)
# plt.ylim(top=100)

# Axis labels
plt.xlabel(r"\textbf{Power [kW]}", fontsize=12, labelpad=12)
plt.ylabel(r"\textbf{Frequency}", fontsize=12, labelpad=12)
plt.grid(True)

# Plot title
plt.title(r"\textbf{Histogram of AC Power Output for Plants 1 and 2} \small(for a 34 days period, excluding zero values)", fontsize=14, pad=10)

# Legend
handles, labels = ax.get_legend_handles_labels()
legend = plt.legend(handles[::1], labels[::1], fontsize=10, title_fontsize=12) # not reversed
legend.get_frame().set_linewidth(0)  # removes border line

# Axis ticks
plt.minorticks_on()
plt.tick_params(which='major', direction='out', width=0.8, bottom=True, left=True, color='black',size=4)  # black ticks

# Draw box around plot area (axes)
for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_edgecolor("black")
    spine.set_linewidth(.5)

# Grid
plt.grid(True, which='minor', linestyle=':', linewidth=0.5, color='white')
plt.grid(True, which='major', linestyle='-', linewidth=0.8, color='white')
plt.tight_layout()

# plt.savefig("/Users/sergimarsol/Documents/UNIVERSITAT/TFG/Thesis/Figures/arrival_delay_histplot.svg", format="svg")
plt.show()


In [ ]:
# Boxplot of AC Power for Plants 1 and 2
# --------------------------------------------
plants_gen['PLANT_ID'] = plants_gen['PLANT_ID'].astype('category')

# Custom X-axis labels
unique_plant_ids = [plant1_gen['PLANT_ID'].unique()[0], plant2_gen['PLANT_ID'].unique()[0]]
custom_labels = ['Plant 1', 'Plant 2']

fig, ax = plt.subplots(figsize=(10, 6))

sns.boxplot(
    x='PLANT_ID',      # X-axis: The categorical variable to group by (the plants)
    y='AC_POWER',      # Y-axis: The continuous variable to plot the distribution of
    data=plants_gen[plants_gen['AC_POWER'] > 0],
    hue = 'PLANT_ID',
    palette=[darkblue, darkcyan],
)

plt.xticks(
    ticks=range(len(unique_plant_ids)), # The positions (0 and 1) where the labels go
    labels=custom_labels,               # The new labels to display ('Plant 1', 'Plant 2')
    fontsize=11
)

# Title
plt.title(r"\textbf{Boxplot of AC Power Output for Plants 1 and 2} \small(for a 34 days period, excluding zero values)", fontsize=14, pad=10)

# Draw box around plot area (axes)
for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_edgecolor("black")
    spine.set_linewidth(.5)

# Hide legend
plt.legend().set_visible(False)

plt.show()

##### A.2 Inversor Level AC Power Output Comparison per Plant (2 plots)

In [ ]:
# Boxplot of AC Power per Inverter, colored by Plant ID
# -------------------------------------------------------
plants_gen['PLANT_ID'] = plants_gen['PLANT_ID'].astype('category')
plants_gen['SOURCE_KEY'] = plants_gen['SOURCE_KEY'].astype('category')

# Filter out zero power readings
df_inverter_analysis = plants_gen[plants_gen['AC_POWER'] > 0].copy()

# Identify unique Plant IDs to ensure the color mapping is correct
unique_plant_ids = df_inverter_analysis['PLANT_ID'].unique()

# Create a custom color map to color inverters based on their PLANT_ID
# (Assuming darkblue and darkcyan are defined variables in your environment)
plant_color_map = {
    unique_plant_ids[0]: darkblue,  # Plant 1 color
    unique_plant_ids[1]: darkcyan   # Plant 2 color
}

fig, ax = plt.subplots(figsize=(24, 8)) # Significantly increased size for readability

# --- 1. Create the Box Plot (x=Inverter, hue=Plant for coloring) ---
sns.boxplot(
    x='SOURCE_KEY',     # X-axis: All Inverters combined
    y='AC_POWER',       # Y-axis: AC Power Output
    data=df_inverter_analysis,
    
    # Grouping and Coloring
    hue='PLANT_ID', 
    palette=plant_color_map, 
    dodge=False,        # Disable dodge so boxes for the same inverter don't overlap (since they are unique across plants)

    # Flier/Outlier Styling
    flierprops={
        'marker': 'o',
        'markerfacecolor': 'red',
        'markeredgecolor': 'red',
        'markersize': 4,
        'alpha': 0.6
    }
)

# --- 2. Manually Set Edge Color and Line Width (Fix for older Seaborn versions) ---
# Set the contour color for all boxes to black (linewidth 0.5)
for box in ax.artists:
    box.set_edgecolor('black')
    box.set_linewidth(0.5)

# Set line properties for whiskers and median
for line in ax.lines:
    line.set_color('black')
    line.set_linewidth(0.8)


# --- 3. Readability Adjustments ---

# Rotate X-axis ticks for readability
plt.xticks(
    rotation=90, # Increased rotation
    ha='right',
    fontsize=8
)

# --- 4. Labels and Title ---

plt.title(r"\textbf{AC Power Distribution by Individual Inverter} \small(for a 34 days period, excluding zero values)", fontsize=16, pad=10)
plt.xlabel(r"\textbf{Inverter ID (SOURCE\_KEY)}", fontsize=12, labelpad=12)
plt.ylabel(r"\textbf{AC Power [kW]}", fontsize=12, labelpad=12)

# Grid
plt.grid(True, which='minor', linestyle=':', linewidth=0.5, color='white')
plt.grid(True, which='major', linestyle='-', linewidth=0.8, color='white')

# Draw box around plot area (axes)
for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_edgecolor("black")
    spine.set_linewidth(.5)

# Add legend and rename labels to Plant 1 and Plant 2
handles, labels = ax.get_legend_handles_labels()
new_labels = ["Plant 1", "Plant 2"]

plt.legend(handles, new_labels, fontsize=10, title_fontsize=12, loc='upper right')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.collections # Needed for type checking if going the collections route, but we stick to artists/lines for compatibility

# Assuming plants_gen, darkblue, and darkcyan are defined in your actual environment
# (The code below assumes their existence and structure as described in your prompt)

# Boxplot of AC Power per Inverter, colored by Plant ID
# -------------------------------------------------------
plants_gen['PLANT_ID'] = plants_gen['PLANT_ID'].astype('category')
plants_gen['SOURCE_KEY'] = plants_gen['SOURCE_KEY'].astype('category')

# Filter out zero power readings
df_inverter_analysis = plants_gen[plants_gen['AC_POWER'] > 0].copy()

# Identify unique Plant IDs to ensure the color mapping is correct
unique_plant_ids = df_inverter_analysis['PLANT_ID'].unique()

# Create a custom color map to color inverters based on their PLANT_ID
# (Assuming darkblue and darkcyan are defined variables in your environment)
plant_color_map = {
    unique_plant_ids[0]: darkblue,  # Plant 1 color
    unique_plant_ids[1]: darkcyan   # Plant 2 color
}

fig, ax = plt.subplots(figsize=(24, 8)) # Significantly increased size for readability

# --- 1. Create the Violin Plot (x=Inverter, hue=Plant for coloring) ---
sns.violinplot(
    x='SOURCE_KEY',      # X-axis: All Inverters combined
    y='AC_POWER',        # Y-axis: AC Power Output
    data=df_inverter_analysis,

    # Grouping and Coloring
    hue='PLANT_ID',
    palette=plant_color_map,
    dodge=False,         # Disable dodge so violins for the same inverter don't overlap (since they are unique across plants)

    # By default, inner='box' will be used, showing a mini boxplot inside the violin
)

# --- 2. Manually Set Edge Color and Line Width ---
# Set the contour color for all violins to black (linewidth 0.5)
for violin in ax.artists:
    violin.set_edgecolor('black')
    violin.set_linewidth(0.5)

# Set line properties for inner lines (e.g., median, quartiles from the inner='box')
for line in ax.lines:
    line.set_color('black')
    line.set_linewidth(0.8)


# --- 3. Readability Adjustments ---

# Rotate X-axis ticks for readability
plt.xticks(
    rotation=90, # Increased rotation
    ha='right',
    fontsize=8
)

# --- 4. Labels and Title ---

# Updated title to reflect the change to Violin Plot
plt.title(r"\textbf{AC Power Distribution (Violin Plot) by Individual Inverter} \small(for a 34 days period, excluding zero values)", fontsize=16, pad=10)
plt.xlabel(r"\textbf{Inverter ID (SOURCE\_KEY)}", fontsize=12, labelpad=12)
plt.ylabel(r"\textbf{AC Power [kW]}", fontsize=12, labelpad=12)

# Grid
plt.grid(True, which='minor', linestyle=':', linewidth=0.5, color='white')
plt.grid(True, which='major', linestyle='-', linewidth=0.8, color='white')

# Draw box around plot area (axes)
for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_edgecolor("black")
    spine.set_linewidth(.5)

# Add legend and rename labels to Plant 1 and Plant 2
handles, labels = ax.get_legend_handles_labels()
new_labels = ["Plant 1", "Plant 2"]

plt.legend(handles, new_labels, fontsize=10, title_fontsize=12, loc='upper right')

plt.show()

#### B. Weather Sensor Data Plots

##### B.1 Solar Irradiance Comparison per Plant (2 plots)

In [ ]:
# Boxplot of Irradiation for Plants 1 and 2
# -------------------------------------------------------
fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(
    x='PLANT_ID',      # X-axis: The categorical variable to group by (the plants)
    y='IRRADIATION',      # Y-axis: The continuous variable to plot the distribution of
    data=plants_weather,
    hue = 'PLANT_ID',
    palette=[darkblue, darkcyan],
)

plt.xticks(
    ticks=range(len(unique_plant_ids)), # The positions (0 and 1) where the labels go
    labels=custom_labels,               # The new labels to display ('Plant 1', 'Plant 2')
    fontsize=11
)


# Title
plt.title(r"\textbf{Boxplot of Irradiation for Plants 1 and 2} \small(for a 34 days period)", fontsize=14, pad=10)
plt.xlabel(r"\textbf{Plant}", fontsize=12, labelpad=12)
plt.ylabel(r"\textbf{Irradiation [kW/m$^\mathbf{2}$]}", fontsize=12, labelpad=12)

# Grid
plt.grid(True, which='minor', linestyle=':', linewidth=0.5, color='white')
plt.grid(True, which='major', linestyle='-', linewidth=0.8, color='white')

# Draw box around plot area (axes)
for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_edgecolor("black")
    spine.set_linewidth(.5)

# Add legend and rename labels to Plant 1 and Plant 2
handles, labels = ax.get_legend_handles_labels()
new_labels = ["Plant 1", "Plant 2"]

legend = plt.legend(handles, new_labels, fontsize=10, title_fontsize=12, loc='upper right')    
legend.get_frame().set_linewidth(0)  # removes border line

plt.show()

In [ ]:
# Violin Plot of Irradiation for Plants 1 and 2
# -------------------------------------------------------
fig, ax = plt.subplots(figsize=(10, 6))

sns.violinplot(
    x='PLANT_ID', 
    y='IRRADIATION', 
    data=plants_weather,
    hue = 'PLANT_ID',
    palette=[darkblue, darkcyan],
    inner='quartile',  # Show median and quartiles inside the violin
    cut=0,             # Prevent violin extending beyond min/max data
    linewidth=1.0,     # Edge width
    edgecolor='black', 
    alpha=0.7,
    ax=ax,
)

# --- 2. Add Stripplot for Outliers/Individual Points ---
sns.stripplot(
    x='PLANT_ID',
    y='IRRADIATION',
    data=plants_weather,
    color='red',      # Use a distinct color (red) to highlight the points
    size=3,           # Small marker size
    alpha=0.2,        # Transparency
    ax=ax
)

# --- 3. X-axis Tick Renaming ---
plt.xticks(
    ticks=range(len(unique_plant_ids)), 
    labels=custom_labels, 
    fontsize=11
)

# --- 4. Labels and Title ---
plt.title(r"\textbf{Violin Plot of Irradiation Distribution by Plant}", fontsize=14, pad=10)
plt.xlabel(r"\textbf{Plant}", fontsize=12, labelpad=12)
plt.ylabel(r"\textbf{Irradiation [kW/m$^\mathbf{2}$]}", fontsize=12, labelpad=12)

# Grid
plt.grid(True, which='minor', linestyle=':', linewidth=0.5, color='white')
plt.grid(True, which='major', linestyle='-', linewidth=0.8, color='white')

# Draw box around plot area (axes)
for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_edgecolor("black")
    spine.set_linewidth(.5)

# Turn legend off
plt.legend().set_visible(False)

# --- 6. Finalization ---
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

##### B.2 Temperature Comparison per Plant (2 plots)

In [ ]:
# Violin Plot of Ambient Temperature for Plants 1 and 2
# -------------------------------------------------------
fig, ax = plt.subplots(figsize=(10, 6))

sns.violinplot(
    x='PLANT_ID', 
    y='AMBIENT_TEMPERATURE', 
    data=plants_weather,
    hue = 'PLANT_ID',
    palette=[darkblue, darkcyan],
    inner='quartile',  # Show median and quartiles inside the violin
    cut=0,             # Prevent violin extending beyond min/max data
    linewidth=1.0,     # Edge width
    edgecolor='black', 
    alpha=0.7,
    ax=ax,
)

# --- 2. Add Stripplot for Outliers/Individual Points ---
sns.stripplot(
    x='PLANT_ID',
    y='AMBIENT_TEMPERATURE',
    data=plants_weather,
    color='red',      # Use a distinct color (red) to highlight the points
    size=3,           # Small marker size
    alpha=0.2,        # Transparency
    ax=ax
)

# --- 3. X-axis Tick Renaming ---
plt.xticks(
    ticks=range(len(unique_plant_ids)), 
    labels=custom_labels, 
    fontsize=11
)

# --- 4. Labels and Title ---
plt.title(r"\textbf{Violin Plot of Ambient Temperature Distribution by Plant}", fontsize=14, pad=10)
plt.xlabel(r"\textbf{Plant}", fontsize=12, labelpad=12)
plt.ylabel(r"\textbf{Ambient Temperature [ºC]}", fontsize=12, labelpad=12)

# Grid
plt.grid(True, which='minor', linestyle=':', linewidth=0.5, color='white')
plt.grid(True, which='major', linestyle='-', linewidth=0.8, color='white')

# Draw box around plot area (axes)
for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_edgecolor("black")
    spine.set_linewidth(.5)

# Turn legend off
plt.legend().set_visible(False)

# --- 6. Finalization ---
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Violin Plot of Module Temperature for Plants 1 and 2
# -------------------------------------------------------
fig, ax = plt.subplots(figsize=(10, 6))

sns.violinplot(
    x='PLANT_ID', 
    y='MODULE_TEMPERATURE', 
    data=plants_weather,
    hue = 'PLANT_ID',
    palette=[darkblue, darkcyan],
    inner='quartile',  # Show median and quartiles inside the violin
    cut=0,             # Prevent violin extending beyond min/max data
    linewidth=1.0,     # Edge width
    edgecolor='black', 
    alpha=0.7,
    ax=ax,
)

# --- 2. Add Stripplot for Outliers/Individual Points ---
sns.stripplot(
    x='PLANT_ID',
    y='MODULE_TEMPERATURE',
    data=plants_weather,
    color='red',      # Use a distinct color (red) to highlight the points
    size=3,           # Small marker size
    alpha=0.2,        # Transparency
    ax=ax
)

# --- 3. X-axis Tick Renaming ---
plt.xticks(
    ticks=range(len(unique_plant_ids)), 
    labels=custom_labels, 
    fontsize=11
)

# --- 4. Labels and Title ---
plt.title(r"\textbf{Violin Plot of Module Temperature Distribution by Plant}", fontsize=14, pad=10)
plt.xlabel(r"\textbf{Plant}", fontsize=12, labelpad=12)
plt.ylabel(r"\textbf{Module Temperature [ºC]}", fontsize=12, labelpad=12)

# Grid
plt.grid(True, which='minor', linestyle=':', linewidth=0.5, color='white')
plt.grid(True, which='major', linestyle='-', linewidth=0.8, color='white')

# Draw box around plot area (axes)
for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_edgecolor("black")
    spine.set_linewidth(.5)

# Turn legend off
plt.legend().set_visible(False)

# --- 6. Finalization ---
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

#### C. Power Generation vs Weather Sensor Data

##### C.1 Power Generation vs Irradiance

(Scatter Plot)

In [ ]:
# Scatter Plot: DC Power vs AC Power Conversion Check
# -------------------------------------------------------

# --- 0. Data Setup and Filtering ---

# Filter data to include only generating periods for clearer visualization of the relationship
df_conversion_check = plants_gen[plants_gen['DC_POWER'] > 0].copy()

# Identify unique Plant IDs for coloring
unique_plant_ids = df_conversion_check['PLANT_ID'].unique()
plant_color_map = {
    unique_plant_ids[0]: darkblue,  # Color for Plant 1
    unique_plant_ids[1]: darkcyan   # Color for Plant 2
}

fig, ax = plt.subplots(figsize=(10, 8))

# --- 1. Create the Scatter Plot ---
sns.scatterplot(
    x='IRRADIATION',
    y='DC_POWER',
    data=plants_df,
    
    # Use Plant ID for color grouping
    hue='PLANT_ID', 
    palette=plant_color_map,
    
    # Styling for points
    s=10,        # Size of the points
    alpha=0.6,   # Transparency for better density visualization
    edgecolor=None # Remove point outlines
)

# --- 2. Labels and Title ---
plt.title(r"\textbf{DC Power vs AC Power Conversion Check}", fontsize=16, pad=10)
plt.xlabel(r"\textbf{DC Power Input [kW]}", fontsize=12, labelpad=12)
plt.ylabel(r"\textbf{AC Power Output [kW]}", fontsize=12, labelpad=12)

# Grid
plt.grid(True, which='minor', linestyle=':', linewidth=0.5, color='white')
plt.grid(True, which='major', linestyle='-', linewidth=0.8, color='white')
plt.tight_layout()

# Draw box around plot area (axes)
for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_edgecolor("black")
    spine.set_linewidth(.5)


# --- 3. Custom Legend ---
# Add legend and rename labels to Plant 1 and Plant 2
handles, labels = ax.get_legend_handles_labels()
new_labels = ["Plant 1", "Plant 2"]


legend = plt.legend(handles, new_labels, fontsize=10, loc='best')
legend.get_frame().set_linewidth(0)  # removes border line


# plt.tight_layout()
plt.show()

### 2.3.3 Trend Analysis

### 2.3.4 Correlation Analysis

In [ ]:
# Scatter Plot: DC Power vs AC Power Conversion Check
# -------------------------------------------------------

# --- 0. Data Setup and Filtering ---

# Filter data to include only generating periods for clearer visualization of the relationship
df_conversion_check = plants_gen[plants_gen['DC_POWER'] > 0].copy()

# Identify unique Plant IDs for coloring
unique_plant_ids = df_conversion_check['PLANT_ID'].unique()
plant_color_map = {
    unique_plant_ids[0]: darkblue,  # Color for Plant 1
    unique_plant_ids[1]: darkcyan   # Color for Plant 2
}

fig, ax = plt.subplots(figsize=(10, 8))

# --- 1. Create the Scatter Plot ---
sns.scatterplot(
    x='DC_POWER',
    y='AC_POWER',
    data=df_conversion_check,
    
    # Use Plant ID for color grouping
    hue='PLANT_ID', 
    palette=plant_color_map,
    
    # Styling for points
    s=10,        # Size of the points
    alpha=0.6,   # Transparency for better density visualization
    edgecolor=None # Remove point outlines
)

# --- 2. Labels and Title ---
plt.title(r"\textbf{DC Power vs AC Power Conversion Check}", fontsize=16, pad=10)
plt.xlabel(r"\textbf{DC Power Input [kW]}", fontsize=12, labelpad=12)
plt.ylabel(r"\textbf{AC Power Output [kW]}", fontsize=12, labelpad=12)

# Grid
plt.grid(True, which='minor', linestyle=':', linewidth=0.5, color='white')
plt.grid(True, which='major', linestyle='-', linewidth=0.8, color='white')
plt.tight_layout()

# Draw box around plot area (axes)
for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_edgecolor("black")
    spine.set_linewidth(.5)


# --- 3. Custom Legend ---
# Add legend and rename labels to Plant 1 and Plant 2
handles, labels = ax.get_legend_handles_labels()
new_labels = ["Plant 1", "Plant 2"]


legend = plt.legend(handles, new_labels, fontsize=10, loc='best')
legend.get_frame().set_linewidth(0)  # removes border line


# plt.tight_layout()
plt.show()

### 2.3.5 Pattern Identification

## 2.4 Feature Engineering

### 2.4.1 Feature Scaling

### 2.4.2 Feature Selection